In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import pandas as pd
import time
from time import mktime, strptime
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm

listofplaces = ["manila","luzon","visayas","mindanao"]
dictt = {"manila manila":3, "manila luzon": 5,"manila visayas": 7,"manila mindanao": 7, "luzon manila":5,"luzon luzon":5,"luzon visayas":7,"luzon mindanao":7,"visayas manila":7,"visayas luzon":7,"visayas visayas":7,"visayas mindanao":7,"mindanao manila":7,"mindanao luzon":7,"mindanao visayas":7,"mindanao mindanao":7}

main_csv = 'drive/My Drive/Logistics/delivery_orders_march.csv'
df = pd.read_csv(main_csv).fillna("blank")
df

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,blank,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.5838e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,blank,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,blank,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,blank,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...
...,...,...,...,...,...,...
3176308,31504087640510,1585821728,1.585978e+09,blank,"mayapis. edeliza JC274 5542 brgy.Dita, Metro M...",Intimate St) compound. Sekiat 2B8 98-B Mel far...
3176309,31504147352227,1585895011,1.586179e+09,1.58623e+09,"Superdome PORKY K307 Libertad, PCGCC sub.bagui...","13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10..."
3176310,31504462290482,1585826951,1.585965e+09,blank,Galay's 1republic 333-18 Diliman 74d Malbay fe...,"Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ..."
3176311,31504851495943,1585891194,1.586230e+09,blank,"#4560, 80south biloca vi!!age KM21, MANRESA, ,...",lrt-2 Consolacion 224-A Paradise 6D 36st champ...


In [3]:
buyer = "Block 2, Lots 2,3,10 & 11, Honest St cor. Determined Street, Calamba Premiere International Park (CPIP), Batino, Calamba, Laguna, Philippines Calamba City Batino Laguna Luzon"
seller = "unit 2 seaviews castles, Tambo, Paranaque City, Metro Manila, Metro Manila"

def findPlace(buyer, seller):
  buyerCountry = ""
  sellerCountry = ""
  for i in listofplaces:
    if (buyer.lower().find(i) != -1):
      buyerCountry = i
    if (seller.lower().find(i) != -1):
      sellerCountry = i
  return dictt.get(buyerCountry + " " + sellerCountry)

# print(findPlace(buyer, seller))

In [4]:
def minusHolDay(time1, time2):
  exdays = [1583625600.0,
 1585094400.0,
 1585526400.0,
 1585612800.0,
 1583020800.0,
 1584230400.0,
 1584835200.0,
 1585440000.0,
 1586044800.0,
 1586649600.0,
 1587254400.0,
 1587859200.0]
  #exdays = [1583625600,,30,31,1,15,22,29]
  count = 0
  #t1 = int(time1.split(" March")[0].strip())
  #t2 = int(time2.split(" March")[0].strip())
  for d in exdays:
    if d<=time2 and d>=time1:
      count+=1
  return count

# print(minusHolDay(1583412300,1583850180))

In [5]:
def secToDateTime(sec):
  return datetime.fromtimestamp(sec).strftime("%d/%m/%Y")
  
def daysBetween(d1, d2):
    d1 = datetime.strptime(d1, "%d/%m/%Y")
    d2 = datetime.strptime(d2, "%d/%m/%Y")
    return abs((d2 - d1).days)

In [6]:
finaldict = {}
for index, row in tqdm(df.iterrows()):
  #print(row['orderid'], row['pick'], row['1st_deliver_attempt'], row['2nd_deliver_attempt'],row['buyeraddress'],row['selleraddress'])
  finaldict[row['orderid']] = 0

  workingDays = findPlace(row['buyeraddress'], row['selleraddress'])
  # print("working days: " + str(workingDays))

  if row['1st_deliver_attempt'] != "blank":
    firstDeliveryDays = daysBetween(secToDateTime(row['pick']),secToDateTime(row['1st_deliver_attempt']))
    # print("firstDeliveryDays: " + str(firstDeliveryDays))
    checkFirst = minusHolDay(row['pick'], row['1st_deliver_attempt'])
    firstDays = firstDeliveryDays - checkFirst

    # check if it exceeds working days
    if (firstDays > workingDays):
      finaldict[row['orderid']] = 1
      continue;

  if row['2nd_deliver_attempt'] != "blank":
    secDeliveryDays = daysBetween(secToDateTime(row['1st_deliver_attempt']),secToDateTime(row['2nd_deliver_attempt']))
    # print("secDeliveryDays: " + str(secDeliveryDays))
    checkSec = minusHolDay(row['1st_deliver_attempt'], row['2nd_deliver_attempt'])
    secDays = secDeliveryDays - checkSec

    # check if it exceeds 3 working days
    if (secDays > 3):
      finaldict[row['orderid']] = 1
      continue;

finaldict

3176313it [16:54, 3130.22it/s]


{2215676524: 0,
 2219624609: 0,
 2220979489: 0,
 2221066352: 0,
 2222478803: 0,
 2222597288: 0,
 2222738456: 0,
 2224695304: 1,
 2224704587: 0,
 2225138267: 0,
 2225861241: 0,
 2226935316: 0,
 2226951056: 0,
 2227381103: 0,
 2227760598: 0,
 2227768858: 0,
 2227771109: 0,
 2227783105: 0,
 2227783217: 0,
 2227790841: 1,
 2227812042: 0,
 2227816770: 1,
 2227817800: 1,
 2227824091: 0,
 2227831996: 1,
 2227839899: 0,
 2227840284: 0,
 2227846339: 1,
 2227847460: 0,
 2227849732: 1,
 2227853544: 0,
 2227853698: 0,
 2227858491: 0,
 2227864372: 0,
 2227865143: 0,
 2227870970: 0,
 2227873008: 0,
 2227874381: 0,
 2227878093: 1,
 2227882334: 0,
 2227889126: 0,
 2227892627: 0,
 2227896956: 0,
 2227899069: 0,
 2227914137: 0,
 2227917424: 0,
 2227931999: 0,
 2227939138: 0,
 2227941125: 0,
 2227941380: 0,
 2227949751: 0,
 2227965096: 0,
 2227970321: 1,
 2227976360: 0,
 2227984356: 0,
 2227988576: 0,
 2227997118: 0,
 2227998015: 0,
 2228026920: 0,
 2228036838: 0,
 2228040329: 1,
 2228044256: 1,
 2228048

In [7]:
import csv
csv_file = 'drive/My Drive/Logistics/Submission.csv'
csv_columns = ['orderid', 'is_late']

try:
  with open(csv_file, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for key, value in finaldict.items():
      writer.writerow({'orderid':key, 'is_late':value})
          
except IOError:
  print("I/O error")